In [1]:
# ! pip install yfinance
# ! pip install MetaTrader5

# https://pypi.org/project/TA-Lib/

In [2]:
from function.preprocess_function import  *

# yfinance

In [3]:
current_datetime = datetime.now()
start_date = '2023-01-01'
end_date = current_datetime.strftime('%Y-%m-%d')

In [4]:
data = yf.download('EURUSD=X', start= start_date, end=end_date, interval='1h')
data = data.reset_index()
data['Datetime'] = pd.to_datetime(data['Datetime']).dt.strftime('%Y-%m-%d %H:%M:%S')
new_columns = {
    'Datetime': 'datetime',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Adj Close': 'adj close',
    'Volume': 'volume'
}

# Использование метода rename для замены имен столбцов
data = data.rename(columns=new_columns)

[*********************100%%**********************]  1 of 1 completed


# data preprocessing

In [5]:
 preprocessor = Preprocessing_stock_data(data)
all_data = preprocessor.all_methods()

In [6]:
all_data

,datetime,open,high,low,close,adj close,volume,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,...,LINEARREG460,LINEARREG_ANGLE460,LINEARREG_INTERCEPT460,LINEARREG_SLOPE460,STDDEV460,TSF460,VAR460,median460,mode460,std460
0,2023-01-02 00:00:00,1.070091,1.071123,1.069862,1.070893,1.070893,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.070893,1.070893,NaN
1,2023-01-02 01:00:00,1.071008,1.071237,1.070091,1.070664,1.070664,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.070778,1.070664,0.000081
2,2023-01-02 02:00:00,1.070435,1.070435,1.069633,1.070435,1.070435,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.070664,1.070435,0.000115
3,2023-01-02 03:00:00,1.070549,1.070893,1.070091,1.070893,1.070893,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.070778,1.070893,0.000094
4,2023-01-02 04:00:00,1.070778,1.071123,1.069862,1.070091,1.070091,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.070664,1.070893,0.000096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5039,2023-10-24 19:00:00,1.059210,1.059322,1.058873,1.059098,1.059098,0,0,0,0,...,1.060157,0.001048,1.051758,0.000018,0.004660,1.060175,0.000022,1.055799,1.057865,0.005064
5040,2023-10-24 20:00:00,1.059210,1.059659,1.058985,1.059547,1.059547,0,0,0,0,...,1.060181,0.001052,1.051756,0.000018,0.004662,1.060199,0.000022,1.055855,1.057865,0.005050
5041,2023-10-24 21:00:00,1.059434,1.059659,1.059210,1.059434,1.059434,0,0,0,0,...,1.060208,0.001056,1.051748,0.000018,0.004665,1.060226,0.000022,1.055855,1.057865,0.005036
5042,2023-10-24 22:00:00,1.059434,1.059883,1.059098,1.059547,1.059547,0,0,0,0,...,1.060223,0.001056,1.051763,0.000018,0.004665,1.060241,0.000022,1.055855,1.057865,0.005022
